In [ ]:
!pip install psycopg2-binary

In [3]:
# Using psycopg2 to connect to the PostgresSQL database
import psycopg2
import pprint

# Connect to the database.
# search_path is set to include mimiciii on the server side.
conn = psycopg2.connect(host="...", port="...", user="...", password="...", database="mimic")
cur = conn.cursor()

# set default mimiciii as the default schema
cur.execute('SET search_path to mimiciii')

In [4]:
def run_query(cur, query):
    cur.execute(query)
    query_result = cur.fetchall()
    col_names = [ col.name for col in cur.description]
    pprint.pprint(col_names)
    pprint.pprint(query_result)

In [5]:
#Hospital mortality (age >= 16) and percent of unit stays 
# Refer to https://github.com/MIT-LCP/mimic-iii-paper/


In [6]:
# Top 10 ICD-9 codes of patients that used ‘Insulin’? What disease is ‘4019’ for?
query = \
"""
with p_on_insulin as
(
  select distinct subject_id as subject_id
  from prescriptions
  where lower(drug) like '%insulin%'
)
select d.icd9_code, d_icd.short_title, count(*)
from p_on_insulin p
  , diagnoses_icd d
  , d_icd_diagnoses d_icd
where 
  p.subject_id = d.subject_id
  and d.icd9_code = d_icd.icd9_code
group by d.icd9_code, d_icd.short_title
order by count(*) desc
limit 10;
"""
run_query(cur, query)

['icd9_code', 'short_title', 'count']
[('4019', 'Hypertension NOS', 15401),
 ('4280', 'CHF NOS', 9993),
 ('42731', 'Atrial fibrillation', 9876),
 ('41401', 'Crnry athrscl natve vssl', 9347),
 ('25000', 'DMII wo cmp nt st uncntr', 8350),
 ('2724', 'Hyperlipidemia NEC/NOS', 6883),
 ('5849', 'Acute kidney failure NOS', 6807),
 ('51881', 'Acute respiratry failure', 5702),
 ('5990', 'Urin tract infection NOS', 4928),
 ('53081', 'Esophageal reflux', 4559)]


In [98]:
# Top 10 Diagnosis-Related Group (DRG) code of patients that used ‘Insulin’, and drg_severity is 3+?
query = \
"""
with p_on_insulin as
(
  select distinct subject_id as subject_id
  from prescriptions
  where lower(drug) like '%insulin%'
)
select d.drg_code, d.description, count(*)
from p_on_insulin p
  , drgcodes d
where 
  p.subject_id = d.subject_id
  and d.drg_severity >= 3
group by d.drg_code, d.description
order by count(*) desc
limit 10;
"""
run_query(cur, query)

['drg_code', 'description', 'count']
[('7204', 'Septicemia & Disseminated Infections', 2172),
 ('1633', 'Cardiac Valve Procedures w/o Cardiac Catheterization', 1033),
 ('0044',
  'Tracheostomy W Long Term Mechanical Ventilation W Extensive Procedure',
  779),
 ('0443', 'Intracranial Hemorrhage', 612),
 ('1334', 'Pulmonary Edema & Respiratory Failure', 604),
 ('1653',
  'Coronary Bypass w/ Cardiac Cath Or Percutaneous Cardiac Procedure',
  604),
 ('0054',
  'Tracheostomy W Long Term Mechanical Ventilation W/O Extensive Procedure',
  592),
 ('1663',
  'Coronary Bypass w/o Cardiac Cath Or Percutaneous Cardiac Procedure',
  585),
 ('7104',
  'Infectious & Parasitic Diseases Including HIV W O.R. Procedure',
  553),
 ('0213', 'Craniotomy Except for Trauma', 530)]


In [99]:
# Top 10 Diagnosis that are involved in hospital mortality?
query = \
"""
select d.icd9_code, d_icd.short_title, count(*)
from admissions a
  , diagnoses_icd d
  , d_icd_diagnoses d_icd
where a.hadm_id=d.hadm_id
  and d.icd9_code = d_icd.icd9_code
  and a.hospital_expire_flag=1
group by d.icd9_code, d_icd.short_title
order by count(*) desc
limit 10;
"""
run_query(cur, query)

['icd9_code', 'short_title', 'count']
[('51881', 'Acute respiratry failure', 2282),
 ('4019', 'Hypertension NOS', 2023),
 ('42731', 'Atrial fibrillation', 1931),
 ('4280', 'CHF NOS', 1884),
 ('5849', 'Acute kidney failure NOS', 1820),
 ('99592', 'Severe sepsis', 1423),
 ('0389', 'Septicemia NOS', 1330),
 ('2762', 'Acidosis', 1167),
 ('25000', 'DMII wo cmp nt st uncntr', 1083),
 ('486', 'Pneumonia, organism NOS', 983)]


In [100]:
# Get all note categories and count of notes
query = \
"""
select category, count(*)
from noteevents
group by category
order by count(*) desc;
"""
run_query(cur, query)

['category', 'count']
[('Nursing/other', 822497),
 ('Radiology', 522279),
 ('Nursing', 223556),
 ('ECG', 209051),
 ('Physician ', 141624),
 ('Discharge summary', 59652),
 ('Echo', 45794),
 ('Respiratory ', 31739),
 ('Nutrition', 9418),
 ('General', 8301),
 ('Rehab Services', 5431),
 ('Social Work', 2670),
 ('Case Management ', 967),
 ('Pharmacy', 103),
 ('Consult', 98)]


In [101]:
# Get number of ‘Discharge summary’ that mentioned ‘blood sugar’ (case insensitive)
query = \
"""
select count(*)
from noteevents
where category='Discharge summary'
  and lower(text) like '%blood sugar%'
group by category;
"""
run_query(cur, query)

['count']
[(4457,)]


In [102]:
# Get the last ‘Discharge summary’ of patients that died in hospital. 
# Show 5 notes from the result.
query = \
"""
with adm_dates as
(
select hadm_id, max(chartdate) as last_chartdate
from noteevents
where category='Discharge summary'
group by hadm_id
	)
	select n.row_id, n.subject_id, n.hadm_id, n.chartdate, n.text from
	noteevents n,
	adm_dates a
	where n.hadm_id=a.hadm_id
	and n.chartdate=a.last_chartdate
	and category='Discharge summary'
	limit 5;
"""
run_query(cur, query)

['row_id', 'subject_id', 'hadm_id', 'chartdate', 'text']
[(42102,
  58526,
  100001,
  datetime.datetime(2117, 9, 17, 0, 0),
  'Admission Date:  [**2117-9-11**]              Discharge Date:   '
  '[**2117-9-17**]\n'
  '\n'
  'Date of Birth:  [**2082-3-21**]             Sex:   F\n'
  '\n'
  'Service: MEDICINE\n'
  '\n'
  'Allergies:\n'
  'Levaquin\n'
  '\n'
  'Attending:[**First Name3 (LF) 2195**]\n'
  'Chief Complaint:\n'
  'nausea, vomiting\n'
  '\n'
  '\n'
  'Major Surgical or Invasive Procedure:\n'
  'none\n'
  '\n'
  'History of Present Illness:\n'
  '35F w/ poorly controlled Type 1 diabetes mellitus w/ neuropathy,\n'
  'nephropathy, HTN, gastroparesis, CKD and retinopathy, recently\n'
  'hospitalized for orthostatic hypotension [**2-3**] autonomic\n'
  'neuropathy [**Date range (1) 25088**]; DKA hospitalizations in [**6-12**] '
  'and [**7-12**], now\n'
  'returning w/ 5d history of worsening nausea, vomiting with\n'
  'coffee-ground emesis, chills, and dyspnea on exertion.  Last\

In [103]:
# For patients (>= 16 years old) who received ‘Insulin’, 
# What is the mean/stddev value of their ‘2339-0’ (loinc_code)? Compare it with people that never received ‘Insulin’. 

query = \
"""
select case when pr.received_insulin is not NULL then true 
  else false end as received_insulin, avg(valuenum), stddev(valuenum)
from d_labitems d
  join labevents l
  on d.itemid=l.itemid
  join patients p
  on l.subject_id=p.subject_id
    left join 
    (
      select distinct subject_id as subject_id, 'Yes' as received_insulin
      from prescriptions
      where lower(drug) like '%insulin%'
    ) pr
  on pr.subject_id=l.subject_id
where d.loinc_code='2339-0'
  and extract(year from age(l.charttime, p.dob))>=16
group by pr.received_insulin;
"""
run_query(cur, query)

['received_insulin', 'avg', 'stddev']
[(True, 137.882760353604, 57.1378020625211),
 (False, 148.931372241912, 63.0068859986106)]


In [104]:
# For patients (>= 16 years old) who received ‘Insulin’,
# What is the mean/stddev of their Creatinine Clearance?  (Cockcroft-Gault CrCl = [(140-age) x (Wt in kg) x (0.85 if female)] / (72 x Cr))

query = \
"""
with cc_values as
(
	with p_weight as
	(
	select subject_id, hadm_id, avg(patientweight) as weight
	from inputevents_mv i
	where patientweight is not null
	group by subject_id, hadm_id
		)
		select case when gender='F' then (140-extract(year from age(a.admittime, p.dob)))*p_weight.weight*0.85/(72*valuenum)
					else (140-extract(year from age(a.admittime, p.dob)))*p_weight.weight/(72*valuenum) end as cc_value
		from labevents l,
			d_labitems d,
			admissions a,
			patients p,
			p_weight
		where d.label = 'Creatinine, Serum'
			and d.itemid=l.itemid
			and p_weight.hadm_id=l.hadm_id
			and p_weight.hadm_id=a.hadm_id
			and p_weight.subject_id=p.subject_id
			and extract(year from age(l.charttime, p.dob))>=16
									 )
select avg(cc_value), stddev(cc_value) 
	from cc_values;
"""
run_query(cur, query)

['avg', 'stddev']
[(87.0845791128899, 46.043955101935)]


In [105]:
# Age distribution for people whose ‘2339-0’ result is abnormal? 

query = \
"""
select extract(year from age(l.charttime, p.dob)) as age, count(*)
from
  patients p join
  labevents l
    on p.subject_id=l.subject_id
  join d_labitems d
    on l.itemid=d.itemid
where 
  d.loinc_code='2339-0'
  and l.flag='abnormal'
group by age
order by age asc;
"""
run_query(cur, query)

['age', 'count']
[(0.0, 137),
 (14.0, 1),
 (15.0, 24),
 (16.0, 72),
 (17.0, 174),
 (18.0, 207),
 (19.0, 238),
 (20.0, 352),
 (21.0, 294),
 (22.0, 495),
 (23.0, 489),
 (24.0, 471),
 (25.0, 444),
 (26.0, 353),
 (27.0, 340),
 (28.0, 449),
 (29.0, 351),
 (30.0, 327),
 (31.0, 543),
 (32.0, 420),
 (33.0, 372),
 (34.0, 594),
 (35.0, 539),
 (36.0, 521),
 (37.0, 845),
 (38.0, 579),
 (39.0, 759),
 (40.0, 1114),
 (41.0, 1132),
 (42.0, 1246),
 (43.0, 1314),
 (44.0, 1359),
 (45.0, 1408),
 (46.0, 1455),
 (47.0, 1554),
 (48.0, 2177),
 (49.0, 2140),
 (50.0, 2185),
 (51.0, 2166),
 (52.0, 2176),
 (53.0, 2393),
 (54.0, 2509),
 (55.0, 2559),
 (56.0, 3068),
 (57.0, 3345),
 (58.0, 3304),
 (59.0, 3421),
 (60.0, 3535),
 (61.0, 3637),
 (62.0, 3500),
 (63.0, 3976),
 (64.0, 3955),
 (65.0, 3700),
 (66.0, 3975),
 (67.0, 4061),
 (68.0, 4061),
 (69.0, 3873),
 (70.0, 4292),
 (71.0, 3611),
 (72.0, 4140),
 (73.0, 4030),
 (74.0, 4344),
 (75.0, 3992),
 (76.0, 4277),
 (77.0, 4234),
 (78.0, 4297),
 (79.0, 3110),
 (80.0, 29

In [106]:
# Export all notes of ‘hadm_id=160489’ into separate .txt files (using Python or Java).

query = \
"""
select row_id, text
from noteevents
where hadm_id=160489;
"""
cur.execute(query)
note_result = cur.fetchall()

for a_note in note_result:
    with open('notes/{}.txt'.format(a_note[0]), 'w') as f:
        f.write(a_note[1])


In [3]:
conn.close()